In [1]:
import nltk
import pandas as pd
import numpy as np
import string
import seaborn as sns
import matplotlib
import plotly as plt
import re
import collections
import math
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import tensorflow as tf


In [90]:
df1 = pd.read_csv("2013_Queensland_Floods_dev.tsv",delimiter='\t')
df2 = pd.read_csv("2013_Queensland_Floods_test.tsv",delimiter='\t')
df3 = pd.read_csv("2013_Queensland_Floods_train.tsv",delimiter='\t')

df1=df1[df1["label"]=="relevant"]
df2=df2[df2["label"]=="relevant"]
df3=df3[df3["label"]=="relevant"]

In [91]:
frames = [df1, df2, df3]

df = pd.concat(frames)

In [92]:
df=df.reset_index(drop=True)
df

,tweet_id,text,label
0,295740276847300608,Queensland in grip of new deadly flood crisis http://t.co/qBt1IBMo,relevant
1,296277240499564544,"Thousands displaced in Queensland, NSW as flood threat continues http://t.co/0tZEpR8t #bigwet",relevant
2,295780049926377472,RT @news_mail: Family of seven among those rescued in Queensland flood crisis http://t.co/VLyPyv6h,relevant
3,296011490039844864,Daring rescue of teenager from flood waters: Australia reels from surging floods http://t.co/Qe5FcKlW,relevant
4,296100680396058625,Photos: Flood water rises in Australia http://t.co/QudYfEh2,relevant
...,...,...,...
5409,295693836733267968,RT @GrillTeam: The Queensland government has set up an information line for the flood crisis.. the number is 1800 173 349 #qldflood,relevant
5410,295679387708313603,"Can we have 5 NEMA staff from Nigeria come to Queensland, Australia to observe how to manage flood & emergencies. 6 helicopters deployed.",relevant
5411,295474897973551104,RT @7NewsBrisbane: Foam from rough waves at Alexandra Headland resembles snow on the Sunshine Coast #staysafe #bigwet #queensland #flood ...,relevant
5412,295718238963249152,RT @abcsouthqld: Master Electricians Australia has a free hotline to help flood homeowners reconnect power supplies once waters have rec ...,relevant


In [93]:
tweets=df["text"]
tw=""
for i in range(0,len(df)):
    tw=tw+str(df["text"][i])
tweets
tw

"Queensland in grip of new deadly flood crisis http://t.co/qBt1IBMoThousands displaced in Queensland, NSW as flood threat continues http://t.co/0tZEpR8t #bigwetRT @news_mail: Family of seven among those rescued in Queensland flood crisis http://t.co/VLyPyv6hDaring rescue of teenager from flood waters: Australia reels from surging floods http://t.co/Qe5FcKlWPhotos: Flood water rises in Australia http://t.co/QudYfEh2RT @Volvo_of_doom: Phew, Queensland can relax, the flood crisis is over, @TonyAbbottMHR has arrived with a flouro vest....@ShpunkyLady66 good to hear this weather has been complete maddness we sunny today but still v windyYoungest's Update::: Photos: Flood water rises in Australia:  http://t.co/nzXUxFODAustralia lurches from fire to flood http://t.co/nCWCoE2QInteresting: Queensland's flood crisis deepens - The Australian: Sydney Morning HeraldQueensland's flo... http://t.co/CChsL9sP Please RTCc @I_amtopsee7 RT @AJEnglish: Flood worsens in eastern Australia http://t.co/YfT2AW6

In [94]:
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

tweets_new=[]

for i in range(0,len(df)):
    tweets_new.append(remove_urls(tweets[i]))

tweets_new[1]

'Thousands displaced in Queensland, NSW as flood threat continues  #bigwet'

In [95]:
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is","It's": "It is","let's": "let us","Let's": "Let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all","There's":"There is",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have","amp":"and"}


In [103]:
#bad_characters=[';', ':', '!', "*",'€°','Ã']
s=[]
s1=[]
for i in range(0,len(tweets_new)):
    s=' '.join(re.sub("(@[A-Za-z0-9_]+)|!|;|:|¡|RT|Õ|Ï|Rt|‡|http|Û|°|#|.com|%|‰|Û_|â|€|°Ã›_|€°Ã›_"," ",tweets_new[i]).split())
    #s=''.join(re.sub("(@[A-Za-z0-9_]+)|!|;|:|¡|RT|Rt|‡|#|http|Û|°|.com|%|‰|Û_|â|€|°Ã›_|€°Ã›_","",s))
    s.replace('€°',"")
    s = re.sub('[_"\-;%()|+&=*%,!?:#$@¤Ã\[\]/]', '', s)
    s=re.sub(r"[^a-zA-Z0-9Ìü.,']+", ' ', s)
    if s in contractions:
        s=contractions[s]
        
     # Format words and remove unwanted characters
    s = re.sub(r'https?:\/\/*[\r\n]*', '', s, flags=re.MULTILINE)
    s = re.sub(r'\<a href', '', s)
    s = re.sub(r'&amp;', 'and ', s) 
    s = re.sub(r'[_"\-;%()|+&=*%,!?:ø#$Ê@ª›¤Ã\[\]/]', '', s)
    s = re.sub(r'<br />', ' ', s)
    s = re.sub('\.\.+', ' ', s)
    s=s.replace("amp", "and")
    #s = re.sub(r'\'', ' ', s)
    
    if s not in s1:
        s1.append(s) 
s1  

['Queensland in grip of new deadly flood crisis',
 'Thousands displaced in Queensland NSW as flood threat continues bigwet',
 'Family of seven among those rescued in Queensland flood crisis',
 'Daring rescue of teenager from flood waters Australia reels from surging floods',
 'Photos Flood water rises in Australia',
 'Phew Queensland can relax the flood crisis is over has arrived with a flouro vest ',
 'good to hear this weather has been plete maddness we sunny today but still v windy',
 "Youngest's Update Photos Flood water rises in Australia",
 'Australia lurches from fire to flood',
 "Interesting Queensland's flood crisis deepens The Australian Sydney Morning HeraldQueensland's flo  Please",
 'Cc Flood worsens in eastern Australia',
 "Australia's flood crisis worsens as death toll rises The deadly flood crisis is escalating across southern Quee ",
 'CNN Photos Flood water rises in Australia',
 'Two days now without power in Ferny Grove',
 '2 DAYS LEFT Make sure your Y Natural Purcha

In [104]:
for i in range(0,len(s1)):
    if not s1[i][0].isupper():
        s1[i][0].upper()
    if s1[i][-1] != '.':
        s1[i]+="." 
    
#s1[8].replace("amp", "and")

s11=[]
for i in range(0,len(s1)):
    if s1[i] not in s11:
        s11.append(s1[i])

In [107]:
len(s11)

3106

In [108]:
with open('cleaning_q.txt', 'w') as f:
    for item in s11:
        f.write("%s\n" % item)

In [109]:
#re.sub("[^A-Za-z0-9.,']+", ' ', tweets_new[0])
t=""
for i in range(0,len(s11)):
    t=t+str(s11[i])+" "
    
#t=clean_text(t)
t

"Queensland in grip of new deadly flood crisis. Thousands displaced in Queensland NSW as flood threat continues bigwet. Family of seven among those rescued in Queensland flood crisis. Daring rescue of teenager from flood waters Australia reels from surging floods. Photos Flood water rises in Australia. Phew Queensland can relax the flood crisis is over has arrived with a flouro vest . good to hear this weather has been plete maddness we sunny today but still v windy. Youngest's Update Photos Flood water rises in Australia. Australia lurches from fire to flood. Interesting Queensland's flood crisis deepens The Australian Sydney Morning HeraldQueensland's flo  Please. Cc Flood worsens in eastern Australia. Australia's flood crisis worsens as death toll rises The deadly flood crisis is escalating across southern Quee . CNN Photos Flood water rises in Australia. Two days now without power in Ferny Grove. 2 DAYS LEFT Make sure your Y Natural Purchase Supports the Queensland Flood Appeal 201

In [110]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

In [111]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word=lemmatizer.lemmatize(word)
            #word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [112]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            if word in stopWords:
                continue
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [113]:
stopWords=set(stopwords.words("english"))
stopWords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [114]:
#“how many sentences contain a word”
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [115]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [116]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [117]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        if count_words_in_sentence!=0:
            sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence
        
    return sentenceValue

In [118]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [119]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [120]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [121]:
def clean_text(text, remove_stopwords = False):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    #text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            tagged_words = nltk.pos_tag(word)
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', '', text)
    text = re.sub(r'&amp;', ' ', text) 
    text = re.sub(r'[_"\-;%()|+&=*%,!?:ø#$Ê@ª›¤Ã\[\]/]', '', text)
    text = re.sub(r'<br />', ' ', text)
    #text = re.sub(r'\'', ' ', text)
    
    return text

In [122]:

text1=[]

for i in range(0,len(s11)):
    text1.append(str(clean_text(s11[i])))
    
 

In [123]:
summ1=[]
# 1 Sentence Tokenize
#sentences = sent_tokenize(text)
total_documents =len(text1)
    
    #print(sentences)
    
   # if total_documents>2:
        
    # 2 Create the Frequency matrix of the words in each sentence.
freq_matrix = _create_frequency_matrix(text1)
    #print(freq_matrix)

    # 3 Calculate TermFrequency and generate a matrix
tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    # 5 Calculate IDF and generate a matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithtotal_documents = len(sentences)m: score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary
summary = _generate_summary(text1, sentence_scores, 1.2* threshold)
    
summ1.append(summary)
        
    #else:
   #     summ.append(text)
#clean_text(summary)    

In [124]:
summ1

[" Cc Flood worsens in eastern Australia. Two days now without power in Ferny Grove. Telcos battle Queensland flood ZDNet The Australian Financial ReviewTelcos battle Queensland floodZDNetSummar . Eagle street pier is gone. flood. impact of floods fresh fruit and vegetable shelves empty in woolworths Townsville Qldfloods. Was good rained a lot though. How was your break Have you started work. Ocean and flood foam from the floods in Australia . Australia Brisbane Ballina Yamba Qldfloods via. we are in the middle of a flood but it is sweltering hot. i hope you choke on a dick australia. Power flickering in Oxley and internet playing up now too. BigWet QldFloods. yea we to far from town to flood but still cut off but we goid. My beautiful hometown in ruins gayndah 2013rebuild floods Now the work begins. I am at school and if you do the follow spree i will flood all of Australia not even joking . Flash Flood Australia Australia EDIS Number FF2013012838017AUSDate time 28012013 04 34 52 UTCE

In [125]:
threshold

0.1847264726018981

In [126]:
sentence_scores

{'Queensland in g': 0.1803061953648093,
 'Thousands displ': 0.15124410223959717,
 'Family of seven': 0.2048422940264138,
 'Daring rescue o': 0.22044004353749788,
 'Photos Flood wa': 0.1470329088228381,
 'Phew Queensland': 0.19830643962773337,
 'good to hear th': 0.20090466531800064,
 "Youngest's Upda": 0.14167059524357398,
 'Australia lurch': 0.20560168215304947,
 'Interesting Que': 0.13705932124615466,
 'Cc Flood worsen': 0.24048564892308177,
 "Australia's flo": 0.1334534005639005,
 'CNN Photos Floo': 0.1470329088228381,
 'Two days now wi': 0.283893685476044,
 '2 DAYS LEFT Mak': 0.15612694874178187,
 'Baby stuffed in': 0.11513999436310302,
 'Drama continues': 0.13313264163061161,
 'News Asia gt Br': 0.12546396907793625,
 "Queensland's fl": 0.17245609431400144,
 'Our thoughts an': 0.1607507405101027,
 'Good luck to ev': 0.14049571469086553,
 'Telcos battle Q': 0.2858301809963493,
 'This is a major': 0.1669604731750135,
 'Fire in brazil ': 0.1804609991948009,
 'Preparing to ev': 0.16216

In [135]:
from nltk import *
import re
import urllib
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize
from bs4 import BeautifulSoup
from nltk.corpus import wordnet
from collections import namedtuple
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import operator
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Digraph
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

def remove_stopword(text):
    cachedStopWords = stopwords.words("english")
    text = ' '.join([word for word in text.split() if word not in cachedStopWords])
    return text
#sentences=s1
f2 = open('result_sent.txt','w')
def textrank(document):
    sentences=clean_text(document)
    sentences=sent_tokenize(sentences)
    #sentence_tokenizer = PunktSentenceTokenizer()
    #sentences = sentence_tokenizer.tokenize(document)
   #f = open('F:/PHD/DATA/PYTHON/sent_tokenized_file.txt','w+')
   # f1 = open('F:/PHD/DATA/PYTHON/processed_file.txt','w')
    print (sentences)
  
    #sentence index tagging
    sentence_mod=[]
    processed_sent=[]
    index=0
    eol = '\n'
	
    for item in sentences:
        lineitem = str(index)+' '+item+eol
        #f.writelines(str(lineitem)) 
        new_item=remove_stopword(item)   #remove stopwords
        #f1.writelines((str(new_item)+eol)) 
        index=index+1
        item += u' '
        item +=str(index)
        sentence_mod.append(item)
        
        new_item +=u' '
        new_item +=str(index)
        processed_sent.append(new_item)
    #f.close()
    
    #f1.close
    #print sentence_mod
    #print processed_sent
    bow_matrix = CountVectorizer().fit_transform(processed_sent)
    
    #print bow_matrix
    
    #f1.write(str(bow_matrix))
    normalized = TfidfTransformer().fit_transform(bow_matrix)
    
    #print (normalized)
 
    similarity_graph = normalized * normalized.T
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    #print(similarity_graph)
    scores = nx.pagerank(nx_graph)
    #nx.draw(nx_graph)
    #plt.show()
    return sorted(((scores[i],s) for i,s in enumerate(processed_sent)), #if scores[i]!=scores[i-1]),
                  reverse=True),sentence_mod,nx_graph

#input_file=open('F:/PHD/DATA/PYTHON/input_file.txt', 'r')
#title=input_file.read()
  
#print title
txt_rnk,sentence_mod,graph=textrank(t)
#print sentence_mod
#print txt_rnk
length = len(txt_rnk)
n=int(length*0.05)
#print "n:------------------->",n
i=0
result=[]
while i< n:
    result.append(txt_rnk[i][1])   
    i=i+1
    
result.sort(key = lambda x: int(x.rsplit(' ',1)[1]))# sort according sentence index #

result_sent=''
result_index=[]
for item in result:
    for k in sentence_mod:
        if item.rsplit(None, 1)[-1]==k.rsplit(None, 1)[-1]:
            result_sent+=k.rsplit(' ', 1)[0]+" " #remove sentence index from the result #
            result_index.append(k.rsplit(' ', 1)[1])

f2.write(result_sent)
f2.close

['Queensland in grip of new deadly flood crisis.', 'Thousands displaced in Queensland NSW as flood threat continues bigwet.', 'Family of seven among those rescued in Queensland flood crisis.', 'Daring rescue of teenager from flood waters Australia reels from surging floods.', 'Photos Flood water rises in Australia.', 'Phew Queensland can relax the flood crisis is over has arrived with a flouro vest .', 'good to hear this weather has been plete maddness we sunny today but still v windy.', "Youngest's Update Photos Flood water rises in Australia.", 'Australia lurches from fire to flood.', "Interesting Queensland's flood crisis deepens The Australian Sydney Morning HeraldQueensland's flo Please.", 'Cc Flood worsens in eastern Australia.', "Australia's flood crisis worsens as death toll rises The deadly flood crisis is escalating across southern Quee .", 'CNN Photos Flood water rises in Australia.', 'Two days now without power in Ferny Grove.', '2 DAYS LEFT Make sure your Y Natural Purchas

<function TextIOWrapper.close()>

In [131]:
#sentences
result_sent

"Queensland in grip of new deadly flood crisis. Photos Flood water rises in Australia. Australia lurches from fire to flood. Australia's flood crisis worsens as death toll rises The deadly flood crisis is escalating across southern Quee . Australia's flood crisis worsens as death toll rises TVNZ Brisbane TimesAustralia's flood crisis worsens as d . Brisbane braces as major Queensland flood crisis unfolds via. Queensland's flood crisis deepens The Australian Sydney Morning HeraldQueensland's flood crisis deepensThe Au . Queensland's flood crisis deepens as death toll rises to three The Australian The Australian Financial Review . Queensland's flood crisis deepens The Australian. Queensland counts flood cost as New South Wales braces for river peaks The Guardian The GuardianQueensland co . Stay inside people weather Australia sydney rain storms floods flood NSW. Queensland's flood crisis deepens The Australian Brisbane News. Queensland flood death toll rises to six A sixth death has been

In [132]:
len(result_sent)

16983

In [134]:
s1

['Queensland in grip of new deadly flood crisis.',
 'Thousands displaced in Queensland NSW as flood threat continues bigwet.',
 'Family of seven among those rescued in Queensland flood crisis.',
 'Daring rescue of teenager from flood waters Australia reels from surging floods.',
 'Photos Flood water rises in Australia.',
 'Phew Queensland can relax the flood crisis is over has arrived with a flouro vest .',
 'good to hear this weather has been plete maddness we sunny today but still v windy.',
 "Youngest's Update Photos Flood water rises in Australia.",
 'Australia lurches from fire to flood.',
 "Interesting Queensland's flood crisis deepens The Australian Sydney Morning HeraldQueensland's flo  Please.",
 'Cc Flood worsens in eastern Australia.',
 "Australia's flood crisis worsens as death toll rises The deadly flood crisis is escalating across southern Quee .",
 'CNN Photos Flood water rises in Australia.',
 'Two days now without power in Ferny Grove.',
 '2 DAYS LEFT Make sure your Y 

In [138]:
#f2 = open('result_sent.txt','w')
with open('flood_summary.txt', 'w') as f:
    for item in sent_tokenize(result_sent):
        f.write("%s\n" % item)

In [2]:
import IPython.nbformat.current as nbf
nb = nbf.read(open('nepal_earthquakes.py', 'r'), 'py')
nbf.write(nb, open('nepal_earthquakes.ipynb', 'w'), 'ipynb')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 23015: character maps to <undefined>